# Chapter 8. Object References, Mutability, and Recycling

## Identity, Equality, and Aliases

In [1]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
lewis is charles

True

In [2]:
id(charles), id(lewis)

(4441314288, 4441314288)

In [3]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [4]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
alex == charles

True

In [5]:
alex is not charles

True

## Copies Are Shallow by Default

List
- shallow copy
    - constructor
    - [:]
    
copy module
- shallow copy: `copy.copy(obj)`
- deep copy: `copy.deepcopy(obj)`

In [7]:
lst1 = [3, [55, 44], (7, 8, 9)]
lst2 = list(lst1)
lst2

[3, [55, 44], (7, 8, 9)]

In [8]:
lst2 == lst1

True

In [9]:
lst2 is lst1

False

In [10]:
lst3 = lst1[:]
lst3

[3, [55, 44], (7, 8, 9)]

In [12]:
lst3 == lst1

True

In [13]:
lst3 is lst1

False

## Deep and Shallow Copies of Arbitrary Objects

In [14]:
class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengeers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [15]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
id(bus1), id(bus2), id(bus3)

(4442831784, 4442831896, 4442832064)

In [16]:
bus1.drop('Bill')
bus2.passengers

['Alice', 'Claire', 'David']

In [17]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(4442822792, 4442822792, 4442637384)

In [18]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

In [20]:
from enum import Enum

class CameraType(Enum):
    LCM = 0
    TPCM = 1

camera_type = CameraType.LCM
camera_type

<CameraType.LCM: 0>

In [22]:
camera_type.value == 0

True

In [24]:
a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [25]:
from copy import deepcopy
c = deepcopy(a)
c

[10, 20, [[...], 30]]

In [32]:
a[2][0]

[10, 20, [[...], 30]]

In [33]:
a[2][0][1]

20

In [34]:
a[2][0][1] = 200
a

[10, 200, [[...], 30]]

In [35]:
c

[10, 20, [[...], 30]]

# Function Parameters as References

In [36]:
def f(a, b):
    a += b
    return a

In [37]:
x = 1
y = 2
f(x, y)

3

In [38]:
x, y

(1, 2)

In [39]:
a = [1, 2]
b = [3, 4]
f(a, b)

[1, 2, 3, 4]

In [40]:
a, b

([1, 2, 3, 4], [3, 4])

In [41]:
t = (10, 20)
u = (30, 40)
f(t, u)

(10, 20, 30, 40)

In [42]:
t, u

((10, 20), (30, 40))

## Mutable Types as Parameter Defaults: Bad Idea
- because each default value is evaluated when the function is **defined**

In [43]:
# Bad example: Mutable as default is danger
class HauntedBus:
    """A bus model haunted by ghost passengers"""
    
    def __init__(self, passengers=[]):
        self.passengers = passengers
    
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [44]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [45]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [46]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [47]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [48]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [49]:
bus2.passengers is bus3.passengers

True

In [50]:
bus1.passengers

['Bill', 'Charlie']

In [51]:
dir(HauntedBus.__init__)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [53]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

In [54]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

## Defensive Programming with Mutable Paramters

In [56]:
# Bad example
class TwilightBus:
    """A bus model that makes passengers vanish"""
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers
    
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [57]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Maya', 'Diana']

In [63]:
# Revision
class TwilightBus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [64]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

['Sue', 'Tina', 'Maya', 'Diana', 'Pat']

# del and Garbage Collection
- The `del` statement deletes names, not objects

In [65]:
import weakref
s1 = {1, 2, 3}
s2 = s1
def bye():
    print('Gone with the wind...')

ender = weakref.finalize(s1, bye)
ender.alive


True

In [66]:
del s1
ender.alive

True

In [67]:
s2 = 'spam'

Gone with the wind...


In [68]:
ender.alive

False

# Weak References
- The presence of references is what keeps an object alive in memory
- A common use case is cache
- Weak references to an object do not increase its reference count
    - the target object is called the `referent`
    - a weak reference does not prevent the reference from being barbage collected
- When build a class that is aware of every one of its instances, should create a class attribute with a `WeakSet` to hold the references to them because the class live as long as the Python process unless you deliberately delete them.

In [69]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)
wref

<weakref at 0x108d22638; to 'set' at 0x108c3cba8>

In [70]:
wref()

{0, 1}

In [71]:
a_set = {2, 3, 4}
wref()

{0, 1}

In [73]:
wref() is None

False

In [74]:
wref() is None

False

## The WeakValueDictionary Skit

In [76]:
class Cheese:
    
    def __init__(self, kind):
        self.kind = kind
    
    def __repr__(self):
        return 'Cheese(%r)' % self.kind

In [87]:
import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]

In [88]:
for cheese in catalog:
    stock[cheese.kind] = cheese

In [89]:
sorted(stock.keys())

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [90]:
del catalog
sorted(stock.keys())

['Parmesan']

In [91]:
del cheese
sorted(stock.keys())

[]

## Limitations of Weak References
- Not every Python object may be the target, or referent, of a weak reference
    - `list`
    - `dict`
    - But plain subclass of either can be referent
- `set` is referent

In [92]:
class MyList(list):
    """list subclass whose instances may be weakly referenced"""
    
a_list = MyList(range(10))

# a_list can be the target of a weak reference
wref_to_a_list = weakref.ref(a_list)

# Tricks Python Plays with Immutables

In [93]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t2 is t1

True

In [96]:
id(t2) == id(t1)

True

In [94]:
t3 = t1[:]
t3 is t1

True

In [95]:
id(t3) == id(t1)

True

In [97]:
t3 = (10, 20)
t1

(1, 2, 3)

In [98]:
t1 = (1, 2, 3)
t3 = (1, 2, 3)
t3 is t1

False

In [99]:
s1 = 'ABC'
s2 = 'ABC'
s2 is s1

True